# A (Possible) Code for Calibrating the Tagger Timing

First, we import the tools we'll need:

In [1]:
import Acqu as aq
import AcquDetector as aqdet
import ROOT
import numpy as np
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Import the data to be analyzed:

In [2]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Load in the detector file for the Tagger:

In [3]:
aqdet.LoadDetectors(['taggerNew.json'])

taggerNew.json


Now we plot the data for the tagger channels and apply a gaussian fit (with an offset) to them (note that onl tbe first 10 channels are done, as it's very slow running to do all the channels at once):

In [61]:
taggerChannels = aqdet.Channels['Tagger'] # number of channels in the tagger (368)

myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")
myFit.SetParameter(1,880)
myFit.SetParameter(2,10)
myFit.SetParNames("constant","mean","stdev","offset")


# histos = [None]*taggerChannels
histos = [None]*10
offsets = [None]*10

# for i in range(taggerChannels):
for i in range(10):
    histos[i] = Hist(1200,300,1500)

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)
    taggerTimes = aqdet.Arrays['Tagger']['Time']
    
    for dat in taggerTimes:
        for time in dat['Time']:
            # for i in range(taggerChannels):
            for i in range(10):
                if (dat['channel']==(i)):
                    histos[i].Fill(time)  
    
aq.runFunction(plotCalTagger,0,50000)

ROOT.enableJSVis()

# c1 = ROOT.TCanvas("my1","Title",1000,(taggerChannels/16)*1000)
# c1.Divide(4,taggerChannels/4,0,0)
c1 = ROOT.TCanvas("my1","Title",1000,2500)
c1.Divide(2,5,0,0)

# for i in range(taggerChannels):
for i in range(10):
    c1.cd(i+1)
    histos[i].SetTitle("Channel "+str(i))
    histos[i].Draw("colz")
    histos[i].Fit('myFit')
    offsets[i] = histos[i].Fit('myFit','SQ').Get().Parameter(3)
c1.Draw()
print(offsets)


KeyboardInterrupt: 

 FCN=1112.81 FROM MIGRAD    STATUS=CONVERGED     255 CALLS         256 TOTAL
                     EDM=1.56246e-06    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  constant     4.84875e+01   2.14651e+00   2.97212e-02  -5.46983e-04
   2  mean         8.80419e+02   4.18637e-01   6.78490e-03  -1.64704e-03
   3  stdev        8.98590e+00   3.18427e-01   4.37407e-03  -3.05239e-04
   4  offset       1.58427e+01   1.27440e-01   2.05147e-03  -1.00111e-02
 FCN=1098.99 FROM MIGRAD    STATUS=CONVERGED      81 CALLS          82 TOTAL
                     EDM=1.61701e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  constant     3.77319e+01   2.04329e+00   2.76550e-02   2.39757e-04
   2  mean         8.79720e+02   

Setting them to the ground?:

In [60]:
bins = 1200
offsets2 = [None]*10
ROOT.enableJSVis()
c2 = ROOT.TCanvas("my2","Title",1000,2500)
c2.Divide(2,5,0,0)

histos2 = [None]*10
histos3 = [None]*10
for i in range(10):
    histos2[i] = Hist(bins,300,1500)
    histos3[i] = Hist(bins,300,1500)

for i in range(10):
    for j in range(bins):
        if (histos[i].GetBinContent(j) - offsets[i] >= 0):
            for k in range(int(histos[i].GetBinContent(j) - offsets[i])):
                histos2[i].Fill(j+300)
                 
    c2.cd(i+1)
    histos2[i].SetTitle("Channel "+str(i))
    histos2[i].Draw("colz")
    myFit.SetParameter(1,880)
    myFit.SetParameter(2,10)
    offsets2[i] = histos2[i].Fit('myFit','SQ').Get().Parameter(3)
    

# if we want to move down again:    
    
#for i in range(10):
#    for j in range(bins):
#        if (histos2[i].GetBinContent(j) - offsets[i] >= 0):
#            for k in range(int(histos2[i].GetBinContent(j) - offsets[i])):
#                histos3[i].Fill(j+300)
#    
#    c2.cd(i+1)
#    histos3[i].SetTitle("Channel "+str(i))
#    histos3[i].Draw("colz")
#    myFit.SetParameter(1,880)
#    myFit.SetParameter(2,10)
    
c2.Draw()


Testing Zone:

In [49]:
taggerChannels = aqdet.Channels['Tagger'] # number of channels in the tagger (368)

bins = 1200
h1 = Hist(bins,300,1500)

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)
    taggerTimes = aqdet.Arrays['Tagger']['Time']
    
    for dat in taggerTimes:
        for time in dat['Time']:
            if (dat['channel']==1):
                h1.Fill(time)
        
    
aq.runFunction(plotCalTagger,0,50000)

ROOT.enableJSVis()

c1 = ROOT.TCanvas("my1","Title",1000,1000)

h1.SetTitle("Channel 1")
h1.GetXaxis().SetTitle("Time")
h1.Draw("colz")
myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")
myFit.SetParameter(1,880)
myFit.SetParameter(2,10)
h1.Fit('myFit')


con = h1.Fit('myFit', 'SQ').Get().Parameter(0)
mean = h1.Fit('myFit', 'SQ').Get().Parameter(1)
stdev = h1.Fit('myFit','SQ').Get().Parameter(2)
offset = h1.Fit('myFit','SQ').Get().Parameter(3)
print(con,mean,stdev,offset)

c1.Draw()

(18.782896336644132, 879.1420867213186, 8.825971735291892, 7.266982750767076)
 FCN=1256.07 FROM MIGRAD    STATUS=CONVERGED     266 CALLS         267 TOTAL
                     EDM=2.3061e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.87829e+01   1.39515e+00   2.04020e-02   4.76754e-05
   2  p1           8.79142e+02   6.89720e-01   1.19156e-02  -7.73695e-05
   3  p2           8.82595e+00   5.41234e-01   7.87004e-03  -4.16183e-05
   4  p3           7.26699e+00   8.62581e-02   1.47449e-03   2.21378e-03


In [51]:
# write this in for every bin i guess

bins = 1200
ROOT.enableJSVis()
c2 = ROOT.TCanvas("my2","Title",1000,1000)

h2 = Hist(bins,300,1500)

for i in range(bins):
    if (h1.GetBinContent(i) - offset >= 0):
        for j in range(int(h1.GetBinContent(i) - offset)):
            h2.Fill(i+300)
    
h2.Draw()
myFit.SetParameter(1,880)
myFit.SetParameter(2,10)
offset2 = h2.Fit('myFit','SQ').Get().Parameter(3)


h3 = Hist(bins,300,1500)

for i in range(bins):
    if (h2.GetBinContent(i) - offset >= 0):
        for j in range(int(h2.GetBinContent(i) - offset)):
            h3.Fill(i+300)


h3.SetTitle("Channel 1")
c2.cd(2)
h3.Draw()
myFit.SetParameter(1,880)
myFit.SetParameter(2,5)
h3.Fit('myFit')
offset3 = h3.Fit('myFit','SQ').Get().Parameter(3)
    
c2.Draw()



 FCN=65.8904 FROM MIGRAD    STATUS=CONVERGED     122 CALLS         123 TOTAL
                     EDM=7.35936e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           7.21564e+00   1.02551e+00   3.02739e-03  -7.93094e-05
   2  p1           8.80102e+02   9.41310e-01   3.70028e-03   5.84054e-05
   3  p2           7.12747e+00   1.06429e+00   2.98724e-03  -1.23147e-04
   4  p3           1.10959e+00   2.98460e-01   9.71400e-04  -7.76621e-05
